In [1]:
grammar = {
    ('S','NP','VP'):0.9,
    ('S','VP'):0.1,
    ('VP','V','NP'):0.5,
    ('VP','V'):0.1,
    ('VP','V','@VP_V'):0.3,
    ('VP','V','PP'):0.1,
    ('@VP_V','NP','PP'):1.0,
    ('NP','NP','NP'):0.1,
    ('NP','NP','PP'):0.2,
    ('NP','N'):0.7,
    ('PP','P','NP'):1.0,
    ('N','people'):0.5,
    ('N','fish'):0.2,
    ('N','tanks'):0.2,
    ('N','rods'):0.1,
    ('V','people'):0.1,
    ('V','fish'):0.6,
    ('V','tanks'):0.3,
    ('P','with'):1.0
}

In [2]:
def CKY(words, grammar):
    score = [[{} for i in range(0,len(words)+1)] for j in range(0,len(words)+1)]
    backpointer = [[{} for i in range(0,len(words)+1)] for j in range(0,len(words)+1)]
    nonterms = list(set([rule[0] for rule in grammar.keys()]))
    
    i = 0
    while i < len(words):
        for A in nonterms:
            if (A,words[i]) in grammar:
                score[i][i+1][A] = grammar[(A,words[i])]
                backpointer[i][i+1][A] = [words[i]]
        
        added = True
        while added == True:
            added = False
            for A in nonterms:
                for B in nonterms:
                    if (A,B) in grammar:
                        if score[i][i+1].get(B,0) > 0:
                            prob = grammar[(A,B)] * score[i][i+1][B]
                            if prob > score[i][i+1].get(A,0):
                                score[i][i+1][A] = prob
                                backpointer[i][i+1][A] = [[i,i+1],[B]]
                                added = True
        i += 1
    
    for span in range(2,len(words)+1):
        for begin in range(0,len(words)-span+1):
            end = begin + span
            for split in range(begin+1,end):
                for A in nonterms:
                    for B in nonterms:
                        for C in nonterms:
                            prob = score[begin][split].get(B,0)*score[split][end].get(C,0)*grammar.get((A,B,C),0)
                            if prob > score[begin][end].get(A,0):
                                score[begin][end][A] = prob
                                backpointer[begin][end][A] = [[[begin,split],[B]],[[split,end],[C]]]
                
                added = True
                while added == True:
                    added = False
                    for A in nonterms:
                        for B in nonterms:
                            prob = grammar.get((A,B),0)*score[begin][end].get(B,0)
                            if prob > score[begin][end].get(A,0):
                                score[begin][end][A] = prob
                                backpointer[begin][end][A] = [[begin,end],[B]]
                                added = True
    
    #retrieving parse using backpointer matrix
    if score[0][len(words)].get('S',0.0) != 0.0:
        backt = backpointer[0][len(words)].get('S',0) #getting indices and chunk tags of lower nodes
        backstring = ['[S']
        loop = ['S']
        def backtrace(backt,backstring,loop):
            if len(str(backt[0][0])) != 1: #loop for multiple lower nodes
                for element in backt:
                    if backt not in words:
                        backstring.append('[' + str(element[1][0]))
                        loop.append(element[1][0])
                        backt = backpointer[element[0][0]][element[0][1]][element[1][0]] #setting backt to indices and chunk tag of lower node
                        backtrace(backt,backstring,loop) #recursion
                        if len(loop) != 0:
                            backstring.append(']' + str(loop.pop())) #popping node
                    else:
                        backstring.append(str(backt[0]))
                if len(loop) != 0:
                    backstring.append(']' + str(loop.pop()))

            else:
                if backt[0] not in words: #loop for single lower node
                    backstring.append('[' + str(backt[1][0]))
                    loop.append(backt[1][0])
                    backt = backpointer[backt[0][0]][backt[0][1]][backt[1][0]]
                    backtrace(backt,backstring,loop)
                else:
                    backstring.append(str(backt[0]))

            return(backstring,loop)

        parse,loop = backtrace(backt,backstring,loop)

        backstr = ' '.join(parse)
        if len(loop) == 1:
            backstr += ' ]S' #ensuring that S is closed
    
        return('The probability of the parse is ' + str(score[0][len(words)].get('S',0.0)) + ' and the parse is ' + backstr + '.')
    
    else:
        return('The probability of the parse is ' + str(0.0) + ' and there is no parse.')

In [3]:
print(CKY(['fish','people','fish','tanks'], grammar))
print(CKY(['people','with','fish','rods','fish','people'], grammar))
print(CKY(['fish','with','fish','fish'], grammar))
print(CKY(['fish','with','tanks','people','fish'], grammar))
print(CKY(['fish','people','with','tanks','fish','people','with','tanks'], grammar))
print(CKY(['fish','fish','fish','fish','fish'], grammar))
print(CKY(['rods','rods','rods','rods'], grammar))

The probability of the parse is 0.00018521999999999996 and the parse is [S [NP [NP [N fish ]N [NP [N people ]N ]NP ]NP [VP [V fish ]V [NP [N tanks ]N ]NP ]VP ]NP ]S.
The probability of the parse is 6.482699999999999e-06 and the parse is [S [NP [NP [NP [N people ]N [PP [P with ]P [NP [N fish ]N ]NP ]PP ]NP ]NP [NP [N rods ]N ]NP ]NP [VP [V fish ]V [NP [N people ]N ]NP ]VP ]S.
The probability of the parse is 0.00021167999999999995 and the parse is [S [NP [NP [N fish ]N [PP [P with ]P [NP [N fish ]N ]NP ]PP ]NP ]NP [VP [V fish ]V ]VP ]S.
The probability of the parse is 2.4695999999999992e-05 and the parse is [S [NP [NP [N fish ]N [PP [P with ]P [NP [N tanks ]N ]NP ]PP ]NP ]NP [VP [V people ]V [NP [N fish ]N ]NP ]VP ]S.
The probability of the parse is 1.0890935999999995e-06 and the parse is [S [NP [NP [N fish ]N [NP [NP [N people ]N [PP [P with ]P [NP [N tanks ]N ]NP ]PP ]NP ]NP ]NP ]NP [VP [V fish ]V [@VP_V [NP [N people ]N [PP [P with ]P [NP [N tanks ]N ]NP ]PP ]NP ]@VP_V ]VP ]S.
The pro